## This notebook is for new data in the form of separate CSV, JSON and or Tiffs 
of cores for Lina Wik and Arne Ö data

### to get d90s go to [difference of 90th percentiles](#d90s)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import utils
import importlib
pd.options.display.max_columns = None
pd.options.display.max_rows = 1000

In [ ]:
#create a little function to change names to the style #_#_#
#could be done with regex but..... when you have a problem, use regex and now you have two problems
def renameImage(astring):
    astring=astring.replace("TMA","")
    pieces1st=astring.split("_")
    tma=pieces1st[0]
    pieces2nd=pieces1st[2].split(".")
    loc1=pieces2nd[1][:-1]
    loc2=pieces2nd[1][-1]
    return f"{tma}_{loc1}_{loc2}"   

In [ ]:
#check for resolution carefully
resolution=0.4961

In [ ]:
#join all the trimmed CSV
#remember to change the mu character
#location of the CSV
loc="locationToYourCSV/"
files= ["TMA6_2_Core1.3C.csv", "TMA6_2_Core1.3D.csv", "TMA6_2_Core1.3F.csv", "TMA6_2_Core1.7D.csv", "TMA6_2_Core1.9B.csv", "TMA6_2_Core1.9D.csv", "TMA6_2_Core1.9F.csv", "TMA6_2_Core1.9H.csv", "TMA6_2_Core1.9I.csv", "TMA7_2_Core1.10B.csv", "TMA7_2_Core1.12C.csv", "TMA7_2_Core1.3F.csv", "TMA7_2_Core1.3H.csv", "TMA7_2_Core1.4B.csv", "TMA7_2_Core1.5G.csv", "TMA7_2_Core1.6H.csv", "TMA7_2_Core1.7C.csv", "TMA7_2_Core1.7F.csv", "TMA7_2_Core1.8E.csv", "TMA7_2_Core1.8I.csv"]
alldf={}
for i in files:
    newname=renameImage(i)
    alldf[i]=pd.read_csv(f"{loc}{i}")
    alldf[i]["Image"]=newname
    alldf[i]["global_id"]=alldf[i].index.values

In [ ]:
result = pd.concat(alldf)
result = result.reset_index()


# Now, are these for training or to go to inference directly?

### If training: continue along. Else if inference: go below to where it says [inference](#Inference)

In [ ]:
cols=[x for x in result.columns]

In [ ]:
# work in image coordinates since we are going to search for things in the image
result["cx"]=result['Centroid X mum']/resolution
result["cy"]=result['Centroid Y mum']/resolution

In [ ]:
combvalues={"GFAP":1,"mutIDH1":2,"NeuroC":4,"TMEM119":8,"IBA1":16,"CD34":32,"MBP":64,"Ki67":128,
                "ambiguous":256}
markers=["GFAP","mutIDH1","NeuroC","TMEM119","IBA1","CD34","MBP","Ki67"]

realcombcolors={1:"#2953b9",129:"#5574be",2:"#ff881f",3:"#d2a93b",6:"#ab8d40",7:"#925824",130:"#c3912d",
                131:"#ab672a",134:"#b67b2d",135:"#6f3909",4:"#ea86d5",8:"#ea2325",24:"#bb4b4b",136:"#bb4b4b",
                152:"#ad6666",16:"#9467bd",144:"#9f89b2",32:"#2699b1",160:"#78c1d0",128:"#35c135"}

celltype={1:"Astrocyte",129:"Astrocyte",2:"Glioma",3:"Glioma",6:"Glioma",7:"Glioma",130:"Glioma",
         131:"Glioma",134:"Glioma",135:"Glioma",4:"Neuron",8:"Microglia",24:"Microglia",
         136:"Microglia",152:"Microglia",16:"Macrophage",144:"Macrophage",32:"Endothelial",
         160:"Endothelial",256:"ambiguous"}

coldict={"Astrocyte":"#2953b9","Glioma":"#ff7f0e","Neuron":"#ea86d5",
         "Microglia":"#ea2325","Macrophage":"#9467bd","Endothelial":"#2699b1","ambiguous":"#aaaaaa"}

In [ ]:
combs=[]
comblengths=[]
colvals=[]
celltypes=[]
maincolors=[]
result["combination"]=0
result["combination_length"]=0
result["colorpercell"]=0
result["CellType"]=0
result["maincolor"]=0

In [ ]:
for i, row in result.iterrows():
    classes=row["Class"]
        
    classes=classes.replace(" ","")
    classes=classes.split(":")
    comb=0
    for c in classes:
        comb+=combvalues[c]
    combs.append(comb)
    comblengths.append(len(classes))
    colvals.append(mycolor(comb))
    if comb in celltype:
        ct=celltype[comb]
        celltypes.append(ct)
        maincolors.append(coldict[ct])
    else:
        ct="ambiguous"
        celltypes.append(ct)
        maincolors.append("#aaaaaa")
        

result["combination"]=combs
result["combination_length"]=comblengths
result["colorpercell"]=colvals
result["CellType"]=celltypes
result["maincolor"]=maincolors

**Now it's ready with labels to save and go get d90s**

In [ ]:
result.to_csv("/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/data/detectionResultsTRimmedCores.csv",index=False)

# Inference
preproccessing for inference, where we have no labels

In [ ]:
importlib.reload(utils)
utils.printColsAsString(result,wrap='"',sep=", ")

In [ ]:
#drop useless columns
dropus=[ "level_0", "level_1", "Name", "Class", "Parent", "ROI", "Nucleus: Area", "Nucleus: Perimeter", "Nucleus: Circularity", "Nucleus: Max caliper", "Nucleus: Min caliper", "Nucleus: Eccentricity", "Nucleus: DAPI mean", "Nucleus: DAPI sum", "Nucleus: DAPI std dev", "Nucleus: DAPI max", "Nucleus: DAPI min", "Nucleus: DAPI range", "Nucleus: IBA1 mean", "Nucleus: IBA1 sum", "Nucleus: IBA1 std dev", "Nucleus: IBA1 max", "Nucleus: IBA1 min", "Nucleus: IBA1 range", "Nucleus: Ki67 mean", "Nucleus: Ki67 sum", "Nucleus: Ki67 std dev", "Nucleus: Ki67 max", "Nucleus: Ki67 min", "Nucleus: Ki67 range", "Nucleus: TMEM119 mean", "Nucleus: TMEM119 sum", "Nucleus: TMEM119 std dev", "Nucleus: TMEM119 max", "Nucleus: TMEM119 min", "Nucleus: TMEM119 range", "Nucleus: NeuroC mean", "Nucleus: NeuroC sum", "Nucleus: NeuroC std dev", "Nucleus: NeuroC max", "Nucleus: NeuroC min", "Nucleus: NeuroC range", "Nucleus: MBP mean", "Nucleus: MBP sum", "Nucleus: MBP std dev", "Nucleus: MBP max", "Nucleus: MBP min", "Nucleus: MBP range", "Nucleus: mutIDH1 mean", "Nucleus: mutIDH1 sum", "Nucleus: mutIDH1 std dev", "Nucleus: mutIDH1 max", "Nucleus: mutIDH1 min", "Nucleus: mutIDH1 range", "Nucleus: CD34 mean", "Nucleus: CD34 sum", "Nucleus: CD34 std dev", "Nucleus: CD34 max", "Nucleus: CD34 min", "Nucleus: CD34 range", "Nucleus: GFAP mean", "Nucleus: GFAP sum", "Nucleus: GFAP std dev", "Nucleus: GFAP max", "Nucleus: GFAP min", "Nucleus: GFAP range", "Nucleus: Autofluorescence mean", "Nucleus: Autofluorescence sum", "Nucleus: Autofluorescence std dev", "Nucleus: Autofluorescence max", "Nucleus: Autofluorescence min", "Nucleus: Autofluorescence range", "Cytoplasm: DAPI mean", "Cytoplasm: DAPI std dev", "Cytoplasm: DAPI max", "Cytoplasm: DAPI min", "Cytoplasm: IBA1 mean", "Cytoplasm: IBA1 std dev", "Cytoplasm: IBA1 max", "Cytoplasm: IBA1 min", "Cytoplasm: Ki67 mean", "Cytoplasm: Ki67 std dev", "Cytoplasm: Ki67 max", "Cytoplasm: Ki67 min", "Cytoplasm: TMEM119 mean", "Cytoplasm: TMEM119 std dev", "Cytoplasm: TMEM119 max", "Cytoplasm: TMEM119 min", "Cytoplasm: NeuroC mean", "Cytoplasm: NeuroC std dev", "Cytoplasm: NeuroC max", "Cytoplasm: NeuroC min", "Cytoplasm: MBP mean", "Cytoplasm: MBP std dev", "Cytoplasm: MBP max", "Cytoplasm: MBP min", "Cytoplasm: mutIDH1 mean", "Cytoplasm: mutIDH1 std dev", "Cytoplasm: mutIDH1 max", "Cytoplasm: mutIDH1 min", "Cytoplasm: CD34 mean", "Cytoplasm: CD34 std dev", "Cytoplasm: CD34 max", "Cytoplasm: CD34 min", "Cytoplasm: GFAP mean", "Cytoplasm: GFAP std dev", "Cytoplasm: GFAP max", "Cytoplasm: GFAP min", "Cytoplasm: Autofluorescence mean", "Cytoplasm: Autofluorescence std dev", "Cytoplasm: Autofluorescence max", "Cytoplasm: Autofluorescence min"]
result.drop(columns=dropus,inplace=True)

In [ ]:
columnsorder=["Image","global_id", "Centroid X mum", "Centroid Y mum", "Cell: Area", "Cell: Perimeter", "Cell: Circularity", "Cell: Max caliper", "Cell: Min caliper", "Cell: Eccentricity", "Cell: DAPI mean", "Cell: DAPI std dev", "Cell: DAPI max", "Cell: DAPI min", "Cell: IBA1 mean", "Cell: IBA1 std dev", "Cell: IBA1 max", "Cell: IBA1 min", "Cell: Ki67 mean", "Cell: Ki67 std dev", "Cell: Ki67 max", "Cell: Ki67 min", "Cell: TMEM119 mean", "Cell: TMEM119 std dev", "Cell: TMEM119 max", "Cell: TMEM119 min", "Cell: NeuroC mean", "Cell: NeuroC std dev", "Cell: NeuroC max", "Cell: NeuroC min", "Cell: MBP mean", "Cell: MBP std dev", "Cell: MBP max", "Cell: MBP min", "Cell: mutIDH1 mean", "Cell: mutIDH1 std dev", "Cell: mutIDH1 max", "Cell: mutIDH1 min", "Cell: CD34 mean", "Cell: CD34 std dev", "Cell: CD34 max", "Cell: CD34 min", "Cell: GFAP mean", "Cell: GFAP std dev", "Cell: GFAP max", "Cell: GFAP min", "Cell: Autofluorescence mean", "Cell: Autofluorescence std dev", "Cell: Autofluorescence max", "Cell: Autofluorescence min", "Nucleus/Cell area ratio"]
result=result[columnsorder]
result

**Now it's ready with labels to save and go get d90s**

In [ ]:
loc="/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/data/new20cores/"
result.to_csv(loc+"new20base.csv",index=False)

# d90s 
difference of 90th percentiles

This code was made specifically for this dataset, expeting namig conventions, and markers and features and image types rom this dataset. If you are using this for your own, you will have to give the right metadata

In [ ]:
import JSONexploreCells
from datetime import datetime
import warnings
import importlib

In [ ]:
coreprocessor=JSONexploreCells.CoreProcessor()

In [ ]:
allimages=["LGG TMA 6_2_Core[1,3,C]_[9486,38010]_component_data.tif","LGG TMA 6_2_Core[1,3,D]_[11391,38050]_component_data.tif","LGG TMA 6_2_Core[1,3,F]_[15042,38050]_component_data.tif","LGG TMA 6_2_Core[1,7,D]_[11153,44876]_component_data.tif","LGG TMA 6_2_Core[1,9,B]_[7224,48091]_component_data.tif","LGG TMA 6_2_Core[1,9,D]_[11074,48329]_component_data.tif","LGG TMA 6_2_Core[1,9,F]_[14884,48567]_component_data.tif","LGG TMA 6_2_Core[1,9,H]_[18694,48567]_component_data.tif","LGG TMA 6_2_Core[1,9,I]_[20599,48607]_component_data.tif","LGG TMA 7_2_Core[1,10,B]_[8467,50972]_component_data.tif","LGG TMA 7_2_Core[1,12,C]_[10530,54623]_component_data.tif","LGG TMA 7_2_Core[1,3,F]_[15762,38475]_component_data.tif","LGG TMA 7_2_Core[1,3,H]_[19540,38629]_component_data.tif","LGG TMA 7_2_Core[1,4,B]_[8268,40415]_component_data.tif","LGG TMA 7_2_Core[1,5,G]_[17833,42161]_component_data.tif","LGG TMA 7_2_Core[1,6,H]_[19698,43828]_component_data.tif","LGG TMA 7_2_Core[1,7,C]_[10213,45693]_component_data.tif","LGG TMA 7_2_Core[1,7,F]_[15888,45614]_component_data.tif","LGG TMA 7_2_Core[1,8,E]_[14070,47461]_component_data.tif","LGG TMA 7_2_Core[1,8,I]_[21643,47202]_component_data.tif"]
alljsons=["LGG TMA 6_2_Core[1,3,C]_[9486,38010]_component_data.json", "LGG TMA 6_2_Core[1,3,D]_[11391,38050]_component_data.json", "LGG TMA 6_2_Core[1,3,F]_[15042,38050]_component_data.json", "LGG TMA 6_2_Core[1,7,D]_[11153,44876]_component_data.json", "LGG TMA 6_2_Core[1,9,B]_[7224,48091]_component_data.json", "LGG TMA 6_2_Core[1,9,D]_[11074,48329]_component_data.json", "LGG TMA 6_2_Core[1,9,F]_[14884,48567]_component_data.json", "LGG TMA 6_2_Core[1,9,H]_[18694,48567]_component_data.json", "LGG TMA 6_2_Core[1,9,I]_[20599,48607]_component_data.json", "LGG TMA 7_2_Core[1,10,B]_[8467,50972]_component_data.json", "LGG TMA 7_2_Core[1,12,C]_[10530,54623]_component_data.json", "LGG TMA 7_2_Core[1,3,F]_[15762,38475]_component_data.json", "LGG TMA 7_2_Core[1,3,H]_[19540,38629]_component_data.json", "LGG TMA 7_2_Core[1,4,B]_[8268,40415]_component_data.json", "LGG TMA 7_2_Core[1,5,G]_[17833,42161]_component_data.json", "LGG TMA 7_2_Core[1,6,H]_[19698,43828]_component_data.json", "LGG TMA 7_2_Core[1,7,C]_[10213,45693]_component_data.json", "LGG TMA 7_2_Core[1,7,F]_[15888,45614]_component_data.json", "LGG TMA 7_2_Core[1,8,E]_[14070,47461]_component_data.json", "LGG TMA 7_2_Core[1,8,I]_[21643,47202]_component_data.json"]  

In [ ]:
coreprocessor.multiprocessing=True
#if images already exist, dont waste time loading the big core image
coreprocessor.loadBigImage=True
coreprocessor.location="/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/data/new20cores/JSON/"
coreprocessor.imagelocation="/media/leslie/Elements/LinaData/new20cores/"
coreprocessor.saveat="/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/smallcellsNew20/"
    

In [ ]:
i=0
for imgf,jsf in zip(allimages,alljsons):
    now = datetime.now() 
    print("now = ", now) 
    print(imgf)
    if i ==0 :
        coreprocessor.loadBigImage=False
    else: 
        coreprocessor.loadBigImage=True
    coreprocessor.jsonFile=jsf
    coreprocessor.imgFile=imgf
    coreprocessor.processCore()
    i+=1
    print(i)

In [ ]:
#Add all d90s csv together
d90scols=["Image", "global_id","geometry", "cx","cy", "12d90s", "13d90s", "14d90s", "15d90s", "16d90s", "17d90s", "18d90s", "23d90s", "24d90s", "25d90s", "26d90s", "27d90s", "28d90s", "34d90s", "35d90s", "36d90s", "37d90s", "38d90s", "45d90s", "46d90s", "47d90s", "48d90s", "56d90s", "57d90s", "58d90s", "67d90s", "68d90s", "78d90s"]
d90scsvs=["6_3_C.csv", "6_3_D.csv", "6_3_F.csv", "6_7_D.csv", "6_9_B.csv", "6_9_D.csv", "6_9_F.csv", "6_9_H.csv", "6_9_I.csv", "7_10_B.csv", "7_12_C.csv", "7_3_F.csv", "7_3_H.csv", "7_4_B.csv", "7_5_G.csv", "7_6_H.csv", "7_7_C.csv", "7_7_F.csv", "7_8_E.csv", "7_8_I.csv"]
location="/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/smallcellsNew20/"


In [ ]:
alld90s=[]

for i in d90scsvs:
    df=pd.read_csv(f"{location}{i}")
    df=df[d90scols]
    alld90s.append(df)
    
d90s = pd.concat(alld90s)
d90s = d90s.reset_index()
d90s.to_csv("/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/data/new20cores/d90sfor20newcores.csv")


## Now verify that they match and hopefully it will be ready to mearge by image and global id

In [ ]:
base=pd.read_csv("/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/data/new20cores/new20base.csv")

In [ ]:
d90s=pd.read_csv("/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/data/new20cores/d90sfor20newcores.csv")

In [ ]:
base.sample()

In [ ]:
d90s.sample()

In [ ]:
base[base["Image"]=="7_12_C"].head(10)

In [ ]:
d90s[d90s["Image"]=="7_12_C"].head(10)

In [ ]:
len(base)

In [ ]:
len(d90s)

In [ ]:
base=base.merge(d90s,on=["Image","global_id"])

In [ ]:
base.sample()

In [ ]:
base.drop(columns=["Unnamed: 0","index"],inplace=True)

In [ ]:
base.to_csv("/home/leslie/Documents/Uppsala/TissueMapsAll/5papers/data/new20cores/allinfoonnewcores.csv")

# Now we are ready for inference! in the next notebook